In [3]:
#from rltensor.app import APPAgent
import pickle
import trader
from trader import BFXClient, Trader
from datetime import datetime
from time import sleep
import importlib
import pandas as pd
import numpy as np
import sqlite3

import sys
sys.path.append('REDACTED/rltrader/pytrade_env/pytrade_env/database')
import trade_history
from trade_history import TradeHistory

# dep: pandas 0.22.0
sys.path.append('REDACTED/rltrader/rltensor/execution/online')
import online_helpers as p

In [4]:
def get_weights_dict(weights, currency_pairs):
    dict_ = {}
    for cp, w in zip(currency_pairs, weights):
        dict_[cp] = w
    return dict_

def get_price_dict(currency_pairs):
    k = 'REDACTED'
    s = 'REDACTED'
    bfx = BFXClient(k, s)
    return bfx.get_last_price(currency_pairs).to_dict()

def period_result(weights, currency_pairs):
    weights_dict = get_weights_dict(weights, currency_pairs)
    price_dict = get_price_dict(currency_pairs)
    current_dt = str(datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"))
    return {'datetime': current_dt, 'weights': weights_dict, 'price': price_dict}

def get_period_end():
    now = [int(i) for i in datetime.now().strftime('%Y %m %d %H %M').split()]
    if (int(now[4]) - 30) < 0:
        now[4] = 30
    else:
        now[3] += 1
        if now[3] == 24:
            now[2] += 1
            now[3] = 0
        now[4] = 0
    y, M, d, h, m = now
    period_end = int(datetime(y, M, d, h, m).timestamp())
    return period_end

def sleep_until_period_end(sleep_buffer = 5):
    period_end = get_period_end()
    now = int(datetime.now().timestamp())
    sleep_time = period_end - now + sleep_buffer
    print('Sleeping until period end (%s seconds).' % sleep_time)
    sleep(sleep_time)

# RLTrader

In [4]:
tickers = []

with (open("REDACTED/ticker1.pkl", "rb")) as openfile:
    
    while True:
        try: tickers.append(pickle.load(openfile))
        except EOFError: break

currency_pairs = tickers[0]

print('\nCurrency Pairs Used in Test:\n')
print(currency_pairs)


Currency Pairs Used in Test:

['USDT_BCH', 'USDT_BTC', 'USDT_DASH', 'USDT_ETC', 'USDT_ETH', 'USDT_LTC', 'USDT_XMR', 'USDT_XRP', 'USDT_ZEC']


## Pre-Retrieve New Data (if needed)

In [1]:
p.just_update_trade_history(currency_pairs)

In [5]:
# df = history.get_processed_charts_from_db()[['open', 'high', 'low', 'close', 'volume', 'currency_pair']].copy()
# df[df.index == df.index.max()]

## Online (Balance Only)

In [2]:
# get weights
latest_weights = p.get_online_weights(currency_pairs)

# balance portfolio
trade = trader.Trader(account = 'REDACTED')
trade.rebalance_portfolio(asset_weights = latest_weights.iloc[-1,:].copy(),
                          fill_order_premium = 0.025,
                          order_minimum = 10)

## Train RL Model

In [6]:
str(datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"))

'2018-02-27 00:17:49'

In [7]:
start = '2017-08-03 00:00:00'
end = '2018-02-26 22:00:00'

load_file_path = "params2016-02-01_00:00:00/model.ckpt"
num_epochs = 1

In [9]:
APPAgent.fit(start, end, load_file_path, num_epochs)

# Live Testing

In [7]:
# del trade, trader
# import trader
# importlib.reload(trader)

In [9]:
results = []

#sleep_until_period_end()

while True:
    
    # update model and get prediction
    APPAgent.update_data()
    APPAgent.update_model(num_epochs = 1)
    pred = APPAgent.predict()
    
    # save results
    current_result = dict(period_result(pred, currency_pairs))
    results.append(current_result)
    print(current_result, '\n\n')
    
    '''
    TODO: SORT THE ASSETS TO BALANCE SO THAT NEGATIVES COME FIRST, AND THEN THE
    HIGHEST POSITIVES (SO WE GET THE MOST OF THE ASSET THAT IS WEIGHED THE MOST)
    '''
    
    if int(datetime.now().strftime('%M')) < 30 and \
       int(datetime.now().strftime('%H')) % 2 == 0 and False:
        
        trade = trader.Trader(account = 'REDACTED')
        trade.rebalance_portfolio(asset_weights = pd.Series(current_result['weights']),
                                  fill_order_premium = 0.01,
                                  order_minimum = 10)
    
    
    if int(datetime.now().strftime('%M')) < 30 and \
       int(datetime.utcnow().strftime('%H')) == 0:
        
        print('\n\n\n\n\n========== REBALANCING PORTFOLIO ==========\n\n\n')
            
        # get Online weights
        latest_weights = p.get_online_weights(currency_pairs)
        
        # balance portfolio
        trade = trader.Trader(account = 'REDACTED')
        trade.rebalance_portfolio(asset_weights = latest_weights,
                                  fill_order_premium = 0.025,
                                  order_minimum = 10)
    
    
    # sleep for a little and then update
    sleep(1200)
    p.just_update_trade_history(all_relevant_pairs, verbose = False)
    
    # and now sleep until the end of the period
    print()
    sleep_until_period_end()
    print('\n\n')

## Recent Account Values

In [3]:
trader.Trader(account = 'REDACTED').get_account_value()

# Export for Reporting

In [8]:
export_date = '2018-04-01'

In [9]:
latest_weights = p.get_online_weights(currency_pairs)

history = TradeHistory(currency_pairs = currency_pairs,
                           time_period = '24h',
                           data_dir = 'REDACTED'
                      )

df = history.get_processed_charts_from_db()[['close', 'currency_pair']].copy()
df = history.reformat_for_online_algo(df, ['close'])

if df.shape == latest_weights.shape:
    
    # export
    weights_fname = 'portfolio_weights_export_%s.csv' % export_date
    close_fname = 'portfolio_close_export_%s.csv' % export_date
    latest_weights.index = df.index
    latest_weights.to_csv(weights_fname)
    df.to_csv(close_fname)
    
    print('successfully exported data')
    
else:
    print('dataframes are not the same shape')
    print('closing prices df shape:', df.shape)
    print('weights df shape:', latest_weights.shape)


=== Getting Trade History ===

Period End: 1536426000
Currency Pair: USDT_BCH
Latest Trade: 1536423575000
API Start: 1536423575000
API End: 1536425183214
Successfully added trades to raw db.

Currency Pair: USDT_XMR
Latest Trade: 1536423181000
API Start: 1536423181000
API End: 1536425186459
Successfully added trades to raw db.

Currency Pair: USDT_DASH
Latest Trade: 1536423713000
API Start: 1536423713000
API End: 1536425189607
Successfully added trades to raw db.

Currency Pair: USDT_NEO
Latest Trade: 1536423682000
API Start: 1536423682000
API End: 1536425192760
Successfully added trades to raw db.

Currency Pair: USDT_EOS
Latest Trade: 1536423729000
API Start: 1536423729000
API End: 1536425196002
Successfully added trades to raw db.

Currency Pair: USDT_SAN
Latest Trade: 1536423396000
API Start: 1536423396000
API End: 1536425199142
Successfully added trades to raw db.

Currency Pair: USDT_IOTA
Latest Trade: 1536423617000
API Start: 1536423617000
API End: 1536425202255
Successfully ad